# BUAD 313 - Spring 2025 - Assignment 1 (100 points)

Notes:
 - You may work in teams of up to 3.  Submit one assignment for the three of you, but please ensure it has all 3 of your names and @usc.edu emails.
 - You must submit your work as a .ipynb file (jupyter notebook). The grader has to be able to run your notebook. Code that doesn't run gets zero points.  You may also submit a .pdf version if you wish.  
 - Use the existing sections below to submit your answers below.  You can add additional Python/markdown cells to describe and explain your solution, but keep it tidy.

The deadline for this assignment is **11:59 PM Pacific Time on Friday February 7, 2024**. Late submissions will not be accepted.

Below are the standard Python packages that we use for optimization models in this course. By running this next Python cell, you will have these packages available to use in all your answers contained in this file.

In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum

## Team Names and Emails:
 <font color="blue">**(Edit this cell)**</font>
 - William Jou: wcjou@usc.edu
 - Bain Higgins: cbhiggin@usc.edu
 - Jiya Valiram: jvaliram@usc.edu

## Question 1 (36 Points)

Trojan Kicks manufactures kick scooters. The firm’s assembly process has four workstations. See the process flow diagram below.

<img src="trojanScootersFlow.png" width="1000" />

*Note that if you do not have the file trojanScootersflow.png in the same folder or directory as this notebook, the image will not display. Displaying the figure is not necessary for receiving full credit on this assignment.*


- **Station A** makes the decks; at this workstation, decks are stamped out, welded, and painted.
- At **Station B**, workers assemble the fork, steer support, and T-handle.
- At **Station C**, workers assemble the wheel, brake, and steering mechanism.
- At **Station D**, workers apply decals and grip tape, and conduct the final functional test.

The firm manufactures three products that compete in different segments of the market—**Trojan** scooters in the premium high-end segment, **Tommy** scooters in the mid-market segment, and  **TK** scooters in the low-end segment.

The decks for TK are outsourced; hence, **Station A produces decks only for two products**—Trojan and Tommy. Processing times (in minutes per unit) and the number of workers at each workstation are given in the table below. In each workstation, each worker works independently on their own scooter. The firm operates **8 hours a day, 5 days a week**.  Notice the last column gives the demand for each type of scooter.

| **Product**          | **Station A** (mins/unit) | **Station B** (mins/unit) | **Station C** (mins/unit) | **Finishing** (mins/unit) | **Demand (scooters/week)** |
|----------------------|-------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| **Trojan**          | 20                      | 15                      | 19                      | 28                      | 150                     |
| **Tommy**           | 16                      | 12                      | 13                      | 22                      | 180                     |
| **TK**              | -                        | 10                      | 11                      | 21                      | 120                     |
| *Number of workers* | **3**                    | **5**                    | **6**                    | **4**                    | **N/A**                 |


Given their different marketing, different scooters earn different revenue:

| **Product** | Trojan | Tommy | TK  |
|------------|--------|--------|-----|
| **Price ($)** | 200    | 150    | 100 |


Your goal is to decide how many scooters of  each type to manufacture in a given week to maximize revenue. Note, you are NOT obligated to serve all demand.

**Teaching Note:  Note, this is a substantively more interesting version of a process analysis problem with multiple product types with different flows through the system and different demands.  The point of this problem is to show you how you can use linear optimization to tackle such process analysis questions**

### Part a) (5 points)
What are the decision variables in this this problem?  (Add a Markdown Cell directly below this cell with your answer).  Describe your decision variables in math and in words, and include units.  

The decision variables are:
- How many of each type of scooter to produce (ie. Trojan, Tommy, or TK)
- let S_Trojan = number of Trojan scooters produced
- let S_Tommy = number of Tommy scooters produced
- let S_TK = number of TK scooters produced


### Part b) (5 points):

What is the objective function for this problem? (Add a markdown cell directly below this oen with your answer.) Describe the objective function both in words and in math.  include units.

The objective function of this problem is to maximize the dollar revenue of Trojan Kicks 

max(Revenue) = max($200 * S_Trojan + $150 * S_Tommy + $100 * S_TK)

### Part c) (10 points)
What are the constraints for this problem? For each constraint, be sure to include a short description (in words) of what the constraint represents.  Your description should make the units clear. You should also relax any integrality constraints (i.e. you **can** make fractional scooters since we're thinking about the long-term steady-state performance of the faculty.)

Station A Constraint: The time it takes to complete the process in station A should be less than or equal to the number of minutes the firm operates per week times the number of workers they have in station A
- 20 minutes * S_Trojan + 16 minutes * S_Tommy <= 3 workers * 2,400 minutes
- Simplified: 20 * S_Trojan + 16 * S_Tommy <= 7,200

Station B Constraint: The time it takes to complete the station B process should be less than or equal to the number of minutes the firm operates per week times the number of workers they have in station B
- 15 minutes * S_Trojan + 12 minutes * S_Tommy + 10 minutes * S_TK <= 5 workers * 2,400 minutes
- Simplified: 15 * S_Trojan + 12 * S_Tommy + 10 * S_TK <= 12,000

Station C Constraint: The time it takes to complete the station C process should be less than or equal to the number of minutes the firm operates per week times the number of workers they have in station C
- 19 minutes * S_Trojan + 13 minutes * S_Tommy + 11 minutes * S_TK <= 6 workers * 2,400 minutes
- Simplified: 19 * S_Trojan + 13 * S_Tommy + 11 * S_TK <= 14,400 

Finishing Constraint: The time it takes to complete the finishing stage process should be less than or equal to the number of minutes the firm operates per week times the number of workers they have in the finishing stage
- 28 minutes * S_Trojan + 22 minutes * S_Tommy + 21 minutes * S_TK <= 4 workers * 2,400 minutes
- Simplified: 28 * S_Trojan + 22 * S_Tommy + 21 * S_TK <= 9,600

Demand Constraints: The firm should not produce more than the demand for each product
- S_Trojan <= 150
- S_Tommy <= 180
- S_TK <= 120

Non-negative Constraints: The firm cannot produce a negative number of scooters
- S_Trojan >= 0
- S_Tommy >= 0
- S_TK >= 0

### Part d) (10 points)
Using your formulation above, code up your model in Gurobi and solve it.  Write the optimal value and optimal solution in a markdown cell direclty below this one.  Be sure to label which is which and what the units are!

Below that markdown cell, include your python code for the model in one or more python cells.  Your code should print out the optimal value and optimal solution from your model as its last step. Code that does not run earns no credit!

Optimal Value: 63857.14286
Optimal Solution:
- S_Trojan = 150
- S_Tommy = 180
- S_TK = 68.5714


In [2]:
# Defining Model
m = Model("TrojanKicks")

# Decision Variables
S_Trojan = m.addVar(vtype=GRB.CONTINUOUS, name="S_Trojan")
S_Tommy = m.addVar(vtype=GRB.CONTINUOUS, name="S_Tommy")
S_TK = m.addVar(vtype=GRB.CONTINUOUS, name="S_TK")

# Setting Objectivve Function
m.setObjective(200 * S_Trojan + 150 * S_Tommy + 100 * S_TK, GRB.MAXIMIZE)

# Setting Constraints
m.addConstr(20 * S_Trojan + 16 * S_Tommy <= 7200, name="StationAConstraint")
m.addConstr(15 * S_Trojan + 12 * S_Tommy + 10 * S_TK <= 12000, name="StationBConstraint")
m.addConstr(19 * S_Trojan + 13 * S_Tommy + 11 * S_TK <= 14400, name="StationCConstraint")
m.addConstr(28 * S_Trojan + 22 * S_Tommy + 21 * S_TK <= 9600, name="FinishingStationConstraint")
m.addConstr(S_Trojan <= 150, name="TrojanDemandConstraint")
m.addConstr(S_Tommy <= 180, name="TommyDemandConstraint")
m.addConstr(S_TK <= 120, name="TKDemandConstraint")
m.addConstr(S_Trojan >= 0, name="TrojanNonNegativityConstraint")
m.addConstr(S_Tommy >= 0, name="TommyNonNegativityConstraint")
m.addConstr(S_TK >= 0, name="TKNonNegativityConstraint")

m.optimize()

print()
print('Optimal Values: ')
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))


Set parameter Username
Set parameter LicenseID to value 2614665


Academic license - for non-commercial use only - expires 2026-01-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 265K, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 10 rows, 3 columns and 17 nonzeros
Model fingerprint: 0xe5e8d769
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.02s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.8571429e+04   1.928571e+02   0.000000e+00      0s
       1    6.3857143e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds (0.00 work units)
Optimal objective  6.385714286e+04

Optimal Values: 
S_Trojan 150
S_Tommy 180
S_TK 68.5714


### part d) (6 points)
What are the tight constraints at optimality?  (Hint: You'd expect that there should be 3 of them...)

How would you use these to communicate the optimal solution to a non-technical stakeholder?

The tight constraints at optimality are the Trojan demand constraint, the Tommy demand constraint, and the finishing station constraint. 

The optimal solution gives us the values that would maximize the revenue function. In other words, it gives us the specific numbers of Trojan, Tommy, and TK scooters that we should produce in order to maximize revenue given the constraints that we have. So to a non-technical stakeholder, I would say that as long as we fufill all of the demand for Trojan scotters and Tommy scooters, and also max out the finishing station, we are able to maximize revenue.



## Question 2 ( 30 Points ):  Fulfillment at "Tamazon"

We'll consider a stylized fulfillment problem, similar to the one Amazon solves every day to meet 2 day shipping guarantees for prime members.  

Tamazon has 3 principal warehouses in Los Angeles described in the table below.  With the debut of the new season of Squid Games, green track suits are far and away the most popular item on the site.  Each of the warehouses has a limited supply of track suits (one size fits most) listed as "Inventory" in the table below.  

| Warehouse     | Latitude | Longitude  | Inventory |
|------------------|----------|------------|-----------|
| Hollywood        | 34.0928  | -118.3287  | 600      |
| Venice           | 33.9850  | -118.4695  | 300      |
| Downtown LA      | 34.0407  | -118.2468  | 200      |

There are hundreds of customers who have ordered the track suits and expect them to be delivered in the next two days.  The "fulfillment problem" is to decide from which warehosue we should ship the tracksuit to each customer in order to minimize total costs.  

To get some intuiton and make things simpler, your data science team clustered customer demand and identified 5 zones where most of the demand comes from.  

| Zone       | Latitude | Longitude  | Demand |
|--------------------|----------|------------|--------|
| Beverly Hills      | 34.0736  | -118.4004  | 250    |
| Santa Monica       | 34.0195  | -118.4912  | 150    |
| Westwood           | 34.0635  | -118.4455  | 200    |
| Silver Lake        | 34.0869  | -118.2702  | 310    |
| Echo Park          | 34.0782  | -118.2606  | 50     |

Shipping costs between places in LA is a bit tricky, especially with LA Traffic.  To make things simpler for now, let's assume that shipping costs per track suit are proportional to the distance between two points.  I did this calculation for you, so you don't have to code it.  The distances (in km) are given in the following table:

| Warehouse \ Zone   | Beverly Hills | Santa Monica | Westwood | Silver Lake | Echo Park |
|-------------|--------------:|-------------:|---------:|------------:|----------:|
| Hollywood   |          6.94 |        17.05 |    11.24 |        5.43 |      6.48 |
| Venice      |         11.73 |         4.33 |     9.00 |       21.58 |     21.86 |
| Downtown LA |         14.62 |        22.64 |    18.48 |        5.57 |      4.36 |

Remember, your goal is to ship track suits from the warehouses to the demand zones at minimal cost.


### Part a) (5 pts)
What are the decision variables in this this problem?  (Add a Markdown Cell directly below this cell with your answer).  Describe your decision variables in math and in words, and include units.  

The decision variables for this problem are:
- The number of tracks suits that are shipped from each warehouse to each demand zone

- Let i_HBH = the inventory shipped from Hollywood to Beverley Hills
- Let i_HSM = the inventory shipped from Hollywood to Santa Monica
- Let i_HW = the inventory shipped from Hollywood to Westwood 
- Let i_HSL = the inventory shipped from Hollywood to Silver Lake
- Let i_HEP = the inventory shipped from Hollywood to Echo Park

- Let i_VBH = the inventory shipped from Venice to Beverley Hills
- Let i_VSM = the inventory shipped from Venice to Santa Monica
- Let i_VW = the inventory shipped from Venice to Westwood 
- Let i_VSL = the inventory shipped from Venice to Silver Lake
- Let i_VEP = the inventory shipped from Venice to Echo Park

- Let i_DBH = the inventory shipped from DTLA to Beverley Hills
- Let i_DSM = the inventory shipped from DTLA to Santa Monica
- Let i_DW = the inventory shipped from DTLA to Westwood 
- Let i_DSL = the inventory shipped from DTLA to Silver Lake
- Let i_DEP = the inventory shipped from DTLA to Echo Park

### Part b) (5 points):

What is the objective function for this problem? (Add a markdown cell directly below this one with your answer.) Describe the objective function both in words and in math.  include units.

 The objective function of this problem is to minimize the cost of shipping from warehouses to demand zones

Min(Cost) = Min(6.94 * i_HBH + 17.05 * i_HSM + 11.24 * i_HW + 5.43 * i_HSL + 6.48 * i_HEP
                + 11.73 * i_VBH + 4.33 * i_VSM + 9.00 * i_VW + 21.58 * i_VSL + 21.86 * i_VEP
                + 14.62 * i_DBH + 22.64 * i_DSM + 18.48 * i_DW + 5.57 * i_DSL + 4.36 * i_DEP)


                

### Part c) (10 points)
What are the constraints for this problem? For each constraint, be sure to include a short description (in words) of what the constraint represents.  Your description should make the units clear. You should also relax any integrality constraints (i.e. you **can** ship fractional track suits.)

Hollywood Warehouse Inventory Constraints:
- The total number of track suits shipped from Hollywood must be less than or equal to 600 (it's inventory)
- i_HBH + i_HSM + i_HW + i_HSL + i_HEP <= 600

Venice Warehouse Constraints:
- The total number of track suits shipped from Venice must be less than or equal to 300 (it's inventory)
- i_VBH + i_VSM + i_VW + i_VSL + i_VEP <= 300

DTLA Warehouse Constraints:
- The total number of track suits shipped from DTLA must be less than or equal to 200 (it's inventory)
- i_DBH + i_DSM + i_DW + i_DSL + i_DEP <= 200

Non Negativity Constraints:
- You cannot ship a negative number of track suits
- i_HBH >= 0 
- i_HSM >= 0 
- i_HW >= 0 
- i_HSL >= 0
- i_HEP >= 0

- i_VBH >= 0 
- i_VSM >= 0 
- i_VW >= 0 
- i_VSL >= 0
- i_VEP >= 0

- i_DBH >= 0 
- i_DSM >= 0 
- i_DW >= 0 
- i_DSL >= 0
- i_DEP >= 0

Demand Constraints:
- The number of track suits shipped to Beverly Hills should not exceed the demand of 250
- i_HBH + i_VBH + i_DBH <= 250
- The number of track suits shipped to Santa Monica should not exceed the demand of 150
- i_HSM + i_VSM + i_DSM <= 150
- The number of track suits shipped to Westwood should not exceed the demand of 200
- i_HW + i_VW + i_DW <= 200
- The number of track suits shipped to Silver lake should not exceed the demand of 310
- i_HSL + i_VSL + i_DSL <= 310
- The number of track suits shipped to Echo Park should not exceed the demand of 50
- i_HEP + i_VEP + i_DEP <= 50

### Part d) (10 points)
Using your formulation above, code up your model in Gurobi and solve it.  Write the optimal value and optimal solution in a markdown cell direclty below this one.  Be sure to label which is which and what the units are!

Below that markdown cell, include your python code for the model in one or more python cells.  Your code should print out the optimal value and optimal solution from your model as its last step. Code that does not run earns no credit!

Optimal Value: $6199.2
- i_HBH = 250 track suits
- i_HSM = 0 track suits
- i_HW = 50 track suits
- i_HSL = 300 track suits
- i_HEP = 0 track suits
- i_VBH = 0 track suits
- i_VSM = 150 track suits
- i_VW = 150 track suits
- i_VSL = 0 track suits
- i_VEP = 0 track suits
- i_DBH = 0 track suits
- i_DSM = 0 track suits
- i_DW = 0 track suits
- i_DSL = 10 track suits
- i_DEP = 50 track suits

In [3]:
m = Model("Tamazon")

# Decision Variables

i_HBH = m.addVar(vtype=GRB.CONTINUOUS, name="i_HBH")
i_HSM = m.addVar(vtype=GRB.CONTINUOUS, name="i_HSM")
i_HW = m.addVar(vtype=GRB.CONTINUOUS, name="i_HW")
i_HSL = m.addVar(vtype=GRB.CONTINUOUS, name="i_HSL")
i_HEP = m.addVar(vtype=GRB.CONTINUOUS, name="i_HEP")

i_VBH = m.addVar(vtype=GRB.CONTINUOUS, name="i_VBH")
i_VSM = m.addVar(vtype=GRB.CONTINUOUS, name="i_VSM")
i_VW = m.addVar(vtype=GRB.CONTINUOUS, name="i_VW")
i_VSL = m.addVar(vtype=GRB.CONTINUOUS, name="i_VSL")
i_VEP = m.addVar(vtype=GRB.CONTINUOUS, name="i_VEP")

i_DBH = m.addVar(vtype=GRB.CONTINUOUS, name="i_DBH")
i_DSM = m.addVar(vtype=GRB.CONTINUOUS, name="i_DSM")
i_DW = m.addVar(vtype=GRB.CONTINUOUS, name="i_DW")
i_DSL = m.addVar(vtype=GRB.CONTINUOUS, name="i_DSL")
i_DEP = m.addVar(vtype=GRB.CONTINUOUS, name="i_DEP")

# Setting Objective Function
m.setObjective(6.94 * i_HBH + 17.05 * i_HSM + 11.24 * i_HW + 5.43 * i_HSL + 6.48 * i_HEP
             + 11.73 * i_VBH + 4.33 * i_VSM + 9.00 * i_VW + 21.58 * i_VSL + 21.86 * i_VEP
             + 14.62 * i_DBH + 22.64 * i_DSM + 18.48 * i_DW + 5.57 * i_DSL + 4.36 * i_DEP, GRB.MINIMIZE)

# Constraints

m.addConstr(i_HBH + i_HSM + i_HW + i_HSL + i_HEP <= 600, name="HollyWoodInventoryConstraint")
m.addConstr(i_VBH + i_VSM + i_VW + i_VSL + i_VEP <= 300, name="VeniceInventoryConstraint")
m.addConstr(i_DBH + i_DSM + i_DW + i_DSL + i_DEP <= 200, name="DTLAInventoryConstraint")

m.addConstr(i_HBH >= 0, name="HollyWoodBHNonNegativityConstraint")
m.addConstr(i_HSM >= 0, name="HollyWoodSMNonNegativityConstraint")
m.addConstr(i_HW >= 0, name="HollyWoodWNonNegativityConstraint")
m.addConstr(i_HSL >= 0, name="HollyWoodSLNonNegativityConstraint")
m.addConstr(i_HEP >= 0, name="HollyWoodEPNonNegativityConstraint")

m.addConstr(i_VBH >= 0, name="VeniceBHNonNegativityConstraint")
m.addConstr(i_VSM >= 0, name="VeniceSMNonNegativityConstraint")
m.addConstr(i_VW >= 0, name="VeniceWNonNegativityConstraint")
m.addConstr(i_VSL >= 0, name="VeniceSLNonNegativityConstraint")
m.addConstr(i_VEP >= 0, name="VeniceEPNonNegativityConstraint")

m.addConstr(i_DBH >= 0, name="DTLABHNonNegativityConstraint")
m.addConstr(i_DSM >= 0, name="DTLASMNonNegativityConstraint")
m.addConstr(i_DW >= 0, name="DTLAWNonNegativityConstraint")
m.addConstr(i_DSL >= 0, name="DTLASLNonNegativityConstraint")
m.addConstr(i_DEP >= 0, name="DTLAEPNonNegativityConstraint")

m.addConstr(i_HBH + i_VBH + i_DBH == 250, name="BHDemandConstraint")
m.addConstr(i_HSM + i_VSM + i_DSM == 150, name="SMDemandConstraint")
m.addConstr(i_HW + i_VW + i_DW == 200, name="WDemandConstraint")
m.addConstr(i_HSL + i_VSL + i_DSL == 310, name="SLDemandConstraint")
m.addConstr(i_HEP + i_VEP + i_DEP == 50, name="EPDemandConstraint")

m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 265K, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 23 rows, 15 columns and 45 nonzeros
Model fingerprint: 0xc8d39a03
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 15 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0858000e+03   6.250000e+00   0.000000e+00      0s
       2    6.1992000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.199200000e+03
i_HBH 250
i_HSM 0
i_HW 50
i_HSL 300
i_HEP 0
i_VBH 0
i_VSM 150
i_VW 150
i_VSL 0
i_VEP 0
i_DBH 0
i_DSM 0
i_DW 0
i_DSL 10
i_DEP

## Question 3 ( Points 34 pts ) : Cooling a Server Farm

This question is meant to push  your modeling abilities.  I believe in you.   Work together.  Think. Struggle.  Grow. :)

One of the unforeseen challenges in scaling up computing has been the massive envergy requirements of server farms and compute clusters.  A big portion of these requirements come from the need to cool the room (so the servers don't overheat).  But as they're working, the compute servers generate heat themselves.  We'll look at a stylized model to think about how to minimize cooling costs for a server farm.  We will focus for simplicity on an 8 hour day (although real farms run around the clock!).  In what follows, you should think like a physicist and interpret "cooling" as "negative heat."

In the absence of any other effects, we would expect that the temperature at the beginning of an hour would be the same as the temperature at the beginning of the previous hour.  We will say that to operate effecively, the server room must remain at or below 25 degrees Celsius at all times.  Notice the day starts at 9 am with a temperature of $22$ degrees Celsius.

That said, there are a few effects we should model:
  - During the  hour, the compute servers will do work.  As they work they generate additional heat.
  - During the hour, the ambient temperature provide additional heat (if it's hotter outside) or reduce the heat (if it's colder outside).  
  - During the hour, any cooling we apply (e.g. by cranking up the AC) will reduce the heat in the room.

In theory, we should model the precise temperature in the server room at all points in time (i.e. continuously).  As an approximation, we're only going to think about the temperature in the room at the beginning of every hour (starting at 9 am, and including 4:00 pm) and also at the end of the day (5:00 pm).  We're also going to assume that any additional heat generated or removed *during* the hour doesn't affect the temperature now, but affects the temperature in the next hour.


#### Server Work

Suppose that the demand for the server farm (in Teraflops/hr or TFLOPS/hr) is as follows:

| Hour | Time  | Server Work (TFLOPs/hr) |
|------|-------|--------------------------------|
| 0    | 9 AM  | 150                            |
| 1    | 10 AM | 250                            |
| 2    | 11 AM | 400                            |
| 3    | 12 PM | 500                            |
| 4    | 1 PM  | 500                            |
| 5    | 2 PM  | 450                            |
| 6    | 3 PM  | 350                            |
| 7    | 4 PM  | 200                            |

The additional heat (measured in Celsius) contributed by working is equal to  $.004$ per TFLOP.  


#### Ambient Temperature
At the same time, the room temperature is affected by the ambient environmental temperature.  Here is a forecasted temperature profile for a typical day:

| Hour | Time  | Ambient Temperature (°C) |
|------|-------|--------------------------|
| 0    | 9 AM  | 22                       |
| 1    | 10 AM | 24                       |
| 2    | 11 AM | 26                       |
| 3    | 12 PM | 28                       |
| 4    | 1 PM  | 30                       |
| 5    | 2 PM  | 31                       |
| 6    | 3 PM  | 30                       |
| 7    | 4 PM  | 28                       |

We will assume that the additional heat added to the room during hour $h$ is equal to
$$
.1 * ( \text{Ambient Temperature}_h- \text{Temperature in Room}_h).
$$
This value can be positive or negative (e.g. if it's cold outside.)  The constant $0.1$ represents the natural heat exchange of the building (depending on insulation, etc.)

#### Cooling Effort
Finally, we can exert effort to cool the room (cranking up the AC). For every kilowatt of Cooling Effort we exert in hour $h$, we expect the temperature to decrease by $0.5$ degrees centigrade.


Your goal is to write a linear optimization problem that minimizes the amount of cooling effort needed to keep the server room at a temperature of at most $25$ degrees Celsius throughout the day. Assume the server room starts at a temperature of $22$  degrees at the beginning of the day.



### Part a) Decision Variables (0 points)
I'm going to help you out a bit to get started.  We're going to us the following decision variables:
- Let $T_h$ be the room temperature (Celsius) in hour $h$, for $h = 0, \ldots, 7$.
- Let $C_h$ be the amount of cooling effort (in kilowatts) we apply in hour $h$, for $h = 0, \ldots, 7$.

So we have $16$ decision variables.  Make sure you count them and understand them.

*Teaching Note:  Note that $T_h$ is a bit weird as a decision-variable.  We don't really "decide" it... We decicde $C_h$ and $T_h$ seems to be a consequence of the thermodynamic equations above.  But just trust me. We're going to need this extra decision-variable to make things work.*

### Part b) (5 points):

What is the objective function for this problem? (Add a markdown cell directly below this one with your answer.) Describe the objective function both in words and in math.  include units.

The objective function of this problem is to minimize the amount of cooling effort needed to keep the server room temperature less than or equal to 25 degrees Celsius throughout the 8 hours of operation

Min(Ch) = Min(C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7)





### Part c) (15 points)
What are the constraints for this problem? For each constraint, be sure to include a short description (in words) of what the constraint represents.  Your description should make the units clear. You should also relax any integrality constraints.  

**Hint:  Can you write an equation using the decision variables that tells you the temperature in the room at hour $1$ in terms of the temperature in hour $0$, the ambient temperature at hour $0$, and the cooling effort you put in at hour $0$?**


The temperature at the next hour is equal to the previous hour's temperature plus the temperature added from the work done in the previous hour, plus the temperature added/subtracted from the ambient temperature minus however much cooling effort was put in the previous hour:
- T_h+1 = T_h + 0.004 * W_h + 0.1(AmbT_h - T_h) - 0.5 * C_h

The maximum temperature the room can be at is 25 degrees Celsius:
- T_h <= 25

The initial temperature at time 0 is 22 degrees Celsius:
- T_0 = 22

Cooling effort cannot be negative:
- Ch >= 0

### Part d) (10 points)
Using your formulation above, code up your model in Gurobi and solve it.  Write the optimal value and optimal solution in a markdown cell directly below this one.  Be sure to label which is which and what the units are!

Below that markdown cell, include your python code for the model in one or more python cells.  Your code should print out the optimal value and optimal solution from your model as its last step. Code that does not run earns no credit!

Optimal Value: 19.332 kilowatts 
Optimal Solution:
- T0 = 22 Degrees Celcius
- T1 = 22.6 Degrees Celcius
- T2 = 23.74 Degrees Celcius
- T3 = 25 Degrees Celcius
- T4 = 25 Degrees Celcius
- T5 = 25 Degrees Celcius
- T6 = 25 Degrees Celcius
- T7 = 25 Degrees Celcius
- C0 = 0 kilowatts 
- C1 = 0 kilowatts 
- C2 = 1.132 kilowatts 
- C3 = 4.6 kilowatts 
- C4 = 5 kilowatts 
- C5 = 4.8 kilowatts 
- C6 = 3.8 kilowatts 
- C7 = 0 kilowatts 

In [4]:
m = Model("CoolingEffort")

# Decision Variables

T0 = m.addVar(vtype=GRB.CONTINUOUS, name="T0")
T1 = m.addVar(vtype=GRB.CONTINUOUS, name="T1")
T2 = m.addVar(vtype=GRB.CONTINUOUS, name="T2")
T3 = m.addVar(vtype=GRB.CONTINUOUS, name="T3")
T4 = m.addVar(vtype=GRB.CONTINUOUS, name="T4")
T5 = m.addVar(vtype=GRB.CONTINUOUS, name="T5")
T6 = m.addVar(vtype=GRB.CONTINUOUS, name="T6")
T7 = m.addVar(vtype=GRB.CONTINUOUS, name="T7")

C0 = m.addVar(vtype=GRB.CONTINUOUS, name="C0")
C1 = m.addVar(vtype=GRB.CONTINUOUS, name="C1")
C2 = m.addVar(vtype=GRB.CONTINUOUS, name="C2")
C3 = m.addVar(vtype=GRB.CONTINUOUS, name="C3")
C4 = m.addVar(vtype=GRB.CONTINUOUS, name="C4")
C5 = m.addVar(vtype=GRB.CONTINUOUS, name="C5")
C6 = m.addVar(vtype=GRB.CONTINUOUS, name="C6")
C7 = m.addVar(vtype=GRB.CONTINUOUS, name="C7")

# Setting Objective Function

m.setObjective(C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7, GRB.MINIMIZE)

# Constraints

# Temperature Balance Constraints
m.addConstr(T0 == 22, name="InitialTemperatureConstraint")
m.addConstr(T1 == T0 + 0.004 * 150 + 0.1 * (22 - T0) - 0.5 * C0, name="Temperature1Constraint")
m.addConstr(T2 == T1 + 0.004 * 250 + 0.1 * (24 - T1) - 0.5 * C1, name="Temperature2Constraint")
m.addConstr(T3 == T2 + 0.004 * 400 + 0.1 * (26 - T2) - 0.5 * C2, name="Temperature3Constraint")
m.addConstr(T4 == T3 + 0.004 * 500 + 0.1 * (28 - T3) - 0.5 * C3, name="Temperature4Constraint")
m.addConstr(T5 == T4 + 0.004 * 500 + 0.1 * (30 - T4) - 0.5 * C4, name="Temperature5Constraint")
m.addConstr(T6 == T5 + 0.004 * 450 + 0.1 * (31 - T5) - 0.5 * C5, name="Temperature6Constraint")
m.addConstr(T7 == T6 + 0.004 * 350 + 0.1 * (30 - T6) - 0.5 * C6, name="Temperature7Constraint")


# Max Temperature Constraints
m.addConstr(T0 <= 25, name="MaxTemperature0Constraint")
m.addConstr(T1 <= 25, name="MaxTemperature1Constraint")
m.addConstr(T2 <= 25, name="MaxTemperature2Constraint")
m.addConstr(T3 <= 25, name="MaxTemperature3Constraint")
m.addConstr(T4 <= 25, name="MaxTemperature4Constraint")
m.addConstr(T5 <= 25, name="MaxTemperature5Constraint")
m.addConstr(T6 <= 25, name="MaxTemperature6Constraint")
m.addConstr(T7 <= 25, name="MaxTemperature7Constraint")

# Non-Negativity Constraints
m.addConstr(C0 >= 0, name="CoolingEffort0NonNegativityConstraint")
m.addConstr(C1 >= 0, name="CoolingEffort1NonNegativityConstraint")
m.addConstr(C2 >= 0, name="CoolingEffort2NonNegativityConstraint")
m.addConstr(C3 >= 0, name="CoolingEffort3NonNegativityConstraint")
m.addConstr(C4 >= 0, name="CoolingEffort4NonNegativityConstraint")
m.addConstr(C5 >= 0, name="CoolingEffort5NonNegativityConstraint")
m.addConstr(C6 >= 0, name="CoolingEffort6NonNegativityConstraint")
m.addConstr(C7 >= 0, name="CoolingEffort7NonNegativityConstraint")

m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))




Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 265K, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 16 columns and 38 nonzeros
Model fingerprint: 0x94594437
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+01]
Presolve removed 24 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9332000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.933200000e+01
T0 22
T1 22.6
T2 23.74
T3 25
T4 25
T5 25
T6 25
T7 25
C0 0
C1 0
C2 1.132
C3 4.6
C4 5
C5 4.8
C6 3.8
C7 0


### part e) (4 points)
In actuality, the price of electricity changes throughout the day.  In particular, it is more expensive in the afternoon when it is hot (because everyone turns on their AC and demand is high but supply is limited.)  Suppose you were given a forecast of the hourly price of electricity (similar to the forecasted temperatures).  How would you incorporate this information into your model to improve it? Does  it affect the decision variables, constraints, or objective? Describe the change in detailed paragraph.  

You do NOT need to reimplement your model.  A well-written description is enough.

If I was given a forecast of the hourly price of electricity, the model could be adjusted to reflect the cost in dollars as opposed to the kilowatts. We could change the objective function to instead reflect the cost in dollars of the electricity and minimize that dollar cost. This new objective function would add a weight Ph (the price of electricity in hour h), to each of the cooling efforts Ch. So the new objective function would be to minimize the sum of Ph * Ch for all 8 hours. However, neither the decision variables nor the constraints are changed with this adjustment, as we do not control the price and the constraints remain the same.